In [143]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta
import altair as alt
from pandas.io.json import json_normalize
import requests
import yahoo_fin.stock_info as yf
from alpha_vantage.fundamentaldata import FundamentalData
import json
from fbprophet import Prophet 

D = 400
date_D_days_ago = datetime.now() - timedelta(days=D)
now = datetime.now()

start_date = date_D_days_ago.strftime('%F')
end_date = now.strftime('%F')

apikey='2HCDHLMKIK99309W'
tech_stocks = ['AAPL', 'PTON', 'TSLA','ADYEY']

portfolio2 = ['CSCO','PTON','AI','TSLA','ADYEY','UBER']

tech_stocks[0]


# Operating CashFlow minus Capital Expenditures


'AAPL'

In [144]:
#Get Data from Yahoo
def StockData(ticker, start_d, end_d):
    data = web.get_data_yahoo(ticker, start = start_d, end = end_d)
    price =  pd.DataFrame(data['Adj Close'])
    
    #volume = pd.DataFrame(data['Volume'])
    return price

In [145]:
def Forecast(df,ticker):
    df = df.fillna(0)
    tit = ''+ticker
    df = df.reset_index()
    df.columns = ['ds','y']
    prophet = Prophet()
    prophet.fit(df)
    future_prices=prophet.make_future_dataframe(periods=365)
    future = prophet.predict(future_prices)
    future = future[['ds','trend','yhat_lower','yhat_upper']]
    return future

In [146]:
price = StockData(portfolio2, start_date, end_date)

In [150]:
def Performance(p):
   
    perform = pd.DataFrame(0, index = p.columns, columns=['performance','trend','yhat_lower','yhat_upper'])
       
    df =(p.resample('W').ffill().pct_change() + 1).cumprod().fillna(0).tail(1)
    perform['performance'] = df.iloc[0,:].apply(pct) 
    
    for i in p.columns:
        f = Forecast(p[i],i)
        f.index = f['ds']
        f = f.drop(['ds'], axis=1)
        f = f[now:]
        print(f.tail(1))
        f = (f.resample('W').ffill().pct_change()+1).cumprod().fillna(0).tail(1).applymap(pct)
        print(f)
        perform.loc[i,'trend'] = f.iloc[0,0]
        perform.loc[i,'yhat_lower'] = f.iloc[0,1]
        perform.loc[i,'yhat_upper'] = f.iloc[0,2]
    return perform

In [151]:
p = Performance(price)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


                trend  yhat_lower  yhat_upper
ds                                           
2022-02-11  78.055708   37.650486  121.297449
              trend yhat_lower yhat_upper
ds                                       
2022-02-13  164.72%     85.96%    252.42%


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


                 trend  yhat_lower  yhat_upper
ds                                            
2022-02-11  327.049793  293.214871   359.96965
              trend yhat_lower yhat_upper
ds                                       
2022-02-13  202.63%    194.45%    207.08%


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


                trend  yhat_lower  yhat_upper
ds                                           
2022-02-11  788.30584  669.625163  906.380092
              trend yhat_lower yhat_upper
ds                                       
2022-02-13  442.12%    432.58%    450.95%


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


                  trend   yhat_lower   yhat_upper
ds                                               
2022-02-11  2621.096646  2183.273163  3112.780375
              trend yhat_lower yhat_upper
ds                                       
2022-02-13  285.55%    253.61%    329.04%


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


                 trend  yhat_lower  yhat_upper
ds                                            
2022-02-11  198.945448  167.479891  230.323051
              trend yhat_lower yhat_upper
ds                                       
2022-02-13  473.84%    494.40%    457.00%
                trend  yhat_lower  yhat_upper
ds                                           
2022-02-11  122.91517   102.02813  143.673736
              trend yhat_lower yhat_upper
ds                                       
2022-02-13  205.99%    190.28%    228.04%


In [152]:
p

,performance,trend,yhat_lower,yhat_upper
Symbols,,,,
CSCO,104.50%,164.72%,85.96%,252.42%
PTON,522.96%,202.63%,194.45%,207.08%
AI,130.92%,442.12%,432.58%,450.95%
TSLA,848.75%,285.55%,253.61%,329.04%
ADYEY,122.14%,473.84%,494.40%,457.00%
UBER,178.51%,205.99%,190.28%,228.04%


In [142]:
p['yhat_upper'].astype(float) - p['yhat_lower'].astype(float)

ValueError: could not convert string to float: '220.50%'

In [138]:
p

,performance,trend,yhat_lower,yhat_upper
Symbols,,,,
AAPL,1.76,1.59,1.00,2.20
PTON,5.23,2.03,1.92,2.06
TSLA,8.49,2.86,2.48,3.28
ADYEY,1.22,4.74,4.91,4.68


In [60]:
t.index = t['ds']

In [62]:
t = t.drop(['ds'], axis=1)

In [82]:
t = t[now:]
dig = lambda x: '{:.2f}'.format(x)
pct = lambda x: '{:.2%}'.format(x)

In [85]:
p2 = (t.resample('W').ffill().pct_change()+1).cumprod().fillna(0).tail(1).applymap(dig)

In [86]:
p2


,trend,yhat_lower,yhat_upper
ds,,,
2022-02-13,1.59,1.02,2.22


In [3]:
df1 = pd.DataFrame({'key' : ['a','b','c'],'value': range(3)}, index = ['a1','x2','a3'])
df2 = pd.DataFrame({'key' : ['x','y','x','v'],'stuff': range(4)}, index = ['a1','a2','a3','a4'])

In [4]:
df1

,key,value
a1,a,0
x2,b,1
a3,c,2


In [5]:
df2

,key,stuff
a1,x,0
a2,y,1
a3,x,2
a4,v,3


In [6]:
df1.combine_first(df2)

,key,stuff,value
a1,a,0.0,0.0
a2,y,1.0,NaN
a3,c,2.0,2.0
a4,v,3.0,NaN
x2,b,NaN,1.0


In [ ]:
df1.index.str.match('a3').any()

In [ ]:
df1.columns.values[0]

In [ ]:
%pip install streamlit-disqus

In [18]:
win = yf.get_day_gainers()
win = win.sort_values(by='Market Cap',ascending = False).head(5).sort_values(by='% Change',ascending = False)

In [19]:
win

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM)
14,CICHF,China Construction Bank Corporation,0.84,0.105,14.29,2.199900e+13,9.271100e+13,2.240200e+11,5.42
36,JDCMF,"JD.com, Inc.",49.35,3.740,8.20,3.113300e+13,7.380000e+13,1.540560e+11,NaN
61,ABNB,"Airbnb, Inc.",211.66,11.780,5.89,5.257000e+06,9.675000e+06,1.264560e+11,NaN
77,PDD,Pinduoduo Inc.,199.61,9.850,5.19,6.608000e+06,9.621000e+06,2.447880e+11,NaN
82,TM,Toyota Motor Corporation,161.56,7.560,4.91,1.400000e+06,3.003660e+14,2.228350e+11,9.88


In [20]:
lose = yf.get_day_losers()
lose = lose.sort_values(by='Market Cap',ascending = False).head(5).sort_values(by='% Change',ascending = True)

In [21]:
lose

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM)
8,DFIFF,Diamond Fields Resources Inc.,0.1588,-0.0219,-12.10,3.884900e+13,6.276200e+13,8.389100e+10,NaN
45,AMKBY,A.P. Møller - Mærsk A/S,10.2700,-0.7200,-6.55,3.165920e+14,1.451850e+14,3.759800e+10,14.17
49,PLTR,Palantir Technologies Inc.,35.7000,-2.4700,-6.47,4.496900e+07,7.249200e+07,6.704200e+10,NaN
72,TSLA,"Tesla, Inc.",804.8200,-44.6400,-5.26,3.526000e+07,4.271500e+07,7.725100e+11,1257.53
80,CMPGY,Compass Group PLC,19.2600,-1.0400,-5.12,5.166030e+14,3.014040e+14,3.417200e+10,20.51


In [22]:
active = yf.get_day_most_active()
active = lose.sort_values(by='Market Cap',ascending = False).head(5)

In [23]:
active

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM)
72,TSLA,"Tesla, Inc.",804.8200,-44.6400,-5.26,3.526000e+07,4.271500e+07,7.725100e+11,1257.53
8,DFIFF,Diamond Fields Resources Inc.,0.1588,-0.0219,-12.10,3.884900e+13,6.276200e+13,8.389100e+10,NaN
49,PLTR,Palantir Technologies Inc.,35.7000,-2.4700,-6.47,4.496900e+07,7.249200e+07,6.704200e+10,NaN
45,AMKBY,A.P. Møller - Mærsk A/S,10.2700,-0.7200,-6.55,3.165920e+14,1.451850e+14,3.759800e+10,14.17
80,CMPGY,Compass Group PLC,19.2600,-1.0400,-5.12,5.166030e+14,3.014040e+14,3.417200e+10,20.51


In [31]:
data[["EPS (TTM)"]] = yf.get_quote_table("aapl")

In [33]:
data["EPS (TTM)"] = yf.get_quote_table("aapl")

In [93]:
performance = (price.resample('W').ffill().pct_change() + 1).cumprod().fillna(0).tail(1)

In [94]:
df = pd.DataFrame(0, index= price.columns, columns =['performance','EPS'])

In [95]:
df['performance'] = performance.tail(1)

In [96]:
df['performance'] = performance.iloc[0,:]

In [97]:
my_EPS = lambda x: yf.get_quote_table(x)["EPS (TTM)"]

In [98]:
for i in df.index:
    EPS = yf.get_quote_table(i)["EPS (TTM)"]
    df.loc[i,'EPS'] = EPS
    
    

In [99]:
df

,performance,EPS
Symbols,,
AAPL,1.759296,3.69
PTON,5.256694,-2.11
TSLA,8.603472,0.64


In [100]:
def Plot_EPS(price):
    performance = (price.resample('W').ffill().pct_change() + 1).cumprod().fillna(0).tail(1)
    df = pd.DataFrame(0, index= price.columns, columns =['performance','EPS'])
    df['performance'] = performance.tail(1)
    df['performance'] = performance.iloc[0,:]

    for i in df.index:
        EPS = yf.get_quote_table(i)["EPS (TTM)"]
        df.loc[i,'EPS'] = EPS



    df = df.reset_index()

    pic = alt.Chart(df).mark_point().encode(alt.X('performance:Q',scale=alt.Scale(zero=False)
        ),y='EPS:Q', 
        color='Symbols:N', 
        size=alt.Size("performance:Q", scale=alt.Scale(range=[0, 1000])), 
        tooltip=['Symbols:N', 'performance:N','EPS:N'])
    return pic

alt.LayerChart(...)

In [107]:
table = yf.get_quote_table('CSCO')

pd.read_html(table, index=0)

TypeError: read_html() got an unexpected keyword argument 'index'

In [108]:
table.values

<function dict.values>